# Binarizacion

In [ ]:
import cv2, os, numpy as np
from math import pi

INPUT_DIR = "fotos"
OUTPUT_DIR = "fotos_binarizadas_limpias_test"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# parámetros
LEFT_MASK_FRAC = 0.08        # enmascara el 8% izquierdo (cámbialo si hay borde brillante)
BLUR_KSIZE = 7               # suavizado para reducir ruido (impar)
OPEN_K = 5                   # apertura morfológica para quitar puntos sueltos
AREA_MIN = 150               # descarta blobs muy pequeños
CIRC_MIN = 0.5               # circularidad mínima 4πA/P^2 (0..1)

def keep_largest_circular(blob_mask):
    cnts, _ = cv2.findContours(blob_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    best = None
    best_score = 0.0
    h, w = blob_mask.shape
    for c in cnts:
        area = cv2.contourArea(c)
        if area < AREA_MIN: 
            continue
        peri = cv2.arcLength(c, True) or 1.0
        circ = (4.0 * pi * area) / (peri * peri)  
        score = area * (0.5 + 0.5 * min(max(circ, 0.0), 1.0))  
        if circ >= CIRC_MIN and score > best_score:
            best = c
            best_score = score
    out = np.zeros((h, w), np.uint8)
    if best is not None:
        cv2.drawContours(out, [best], -1, 255, thickness=-1)
    return out

for subdir, _, files in os.walk(INPUT_DIR):
    rel = os.path.relpath(subdir, INPUT_DIR)
    out_sub = os.path.join(OUTPUT_DIR, rel)
    os.makedirs(out_sub, exist_ok=True)

    for f in files:
        if not f.lower().endswith(".jpg"):
            continue
        in_path  = os.path.join(subdir, f)
        out_path = os.path.join(out_sub, f)

        img = cv2.imread(in_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print("⚠️ No se pudo leer", in_path); continue

        h, w = img.shape

        if LEFT_MASK_FRAC > 0:
            m = int(w * LEFT_MASK_FRAC)
            bg = int(img[:, m:m+20].mean()) if m+20 < w else int(img.mean())
            img[:, :m] = bg

        blur = cv2.GaussianBlur(img, (BLUR_KSIZE, BLUR_KSIZE), 0)

        _, bin0 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (OPEN_K, OPEN_K))
        opened = cv2.morphologyEx(bin0, cv2.MORPH_OPEN, k, iterations=1)

        clean = keep_largest_circular(opened)

        cv2.imwrite(out_path, clean)
        print(f" {in_path} -> {out_path}")

print("✨ Proceso completado")

# Mirroring de Imagenes

In [ ]:
import cv2
from pathlib import Path

def mirror_images_with_prefix(base_dir: str, exts={".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}):
    base = Path(base_dir)

    for folder in base.iterdir():
        if not folder.is_dir():
            continue

        out_folder = base / f"in_{folder.name}"
        out_folder.mkdir(parents=True, exist_ok=True)

        for file in folder.glob("*"):
            if file.suffix.lower() not in exts:
                continue

            img = cv2.imread(str(file))
            if img is None:
                continue

            mirrored = cv2.flip(img, 1)

            out_path = out_folder / file.name
            cv2.imwrite(str(out_path), mirrored)

        print(f" Carpeta procesada: {folder.name} -> {out_folder.name}")


BASE_DIR = "fotos_binarizadas_limpias"
mirror_images_with_prefix(BASE_DIR)

# Renombrar un Directorio

In [ ]:
from pathlib import Path

carpeta = Path("./fotos_binarizadas_limpias_test")
prefijo = "lst_"

cont = 0
for p in sorted(carpeta.rglob("*"), key=lambda x: len(str(x)), reverse=True):
    if (
        p.is_dir()
        and not p.name.startswith(".")       
        and not p.name.startswith(prefijo)  
    ):
        destino = p.with_name(prefijo + p.name)
        if destino.exists():
            print(f"⚠️ Ya existe: {destino.name}, salto")
            continue
        p.rename(destino)
        cont += 1
        print(f"📁 {p.name} -> {destino.name}")

print(f" Carpetas renombradas: {cont}")

# Numero de Lanzamientos Disponibles

In [1]:
import os

INPUT_DIR = "fotos_binarizadas_limpias"

subfolders = [d for d in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, d))]

print(f" Número de Lanzamientos: {len(subfolders)}")


 Número de Lanzamientos: 1200
